In [10]:
import pandas as pd
import numpy as np
# import tensorflow as tf
import math
# import tensorflow_datasets as tfds

## Loading the dataset

In [1]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

NameError: name 'tfds' is not defined

In [11]:
# Parameters
image_train_time = 350

min_frequency = 1
max_frequency = 50

In [13]:
def encode_image_to_spike_train(image: np.ndarray):
    spike_trains = []

    for x_position in range(image.shape[0]):
        for y_position in range(image.shape[1]):

            pixel_value = image[x_position][y_position]

            spike_train = np.zeros(shape=(image_train_time,))

            # Transfer pixel value to set frequency range(and some other stuff, which interp does...)
            frequency = np.interp(pixel_value, [np.min(image),np.max(image)], [min_frequency,max_frequency])

            spike_time_distance = math.ceil(image_train_time / frequency)
            next_spike_time = spike_time_distance

            if pixel_value > 0:
                while next_spike_time < image_train_time + 1:
                    # Add Spike to Spike Train
                    spike_train[int(next_spike_time)] = 1

                    # Calculate next spike
                    next_spike_time += spike_time_distance

            spike_trains.append(spike_train)

    return spike_trains


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 27)

In [14]:
def receptive_field(image: np.ndarray):
    image_size_x = image.shape[0]
    image_size_y = image.shape[1]

    weight1 =  0.625
    weight2 =  0.125
    weight3 = -0.125
    weight4 = -.5

    # Receptive Field Kernel
    receptive_field = [
        [weight4 ,weight3 , weight2 ,weight3 ,weight4],
        [weight3 ,weight2 , weight1 ,weight2 ,weight3],
        [ weight2 ,weight1 ,   1    ,weight1 ,weight2],
        [weight3 ,weight2 , weight1 ,weight2 ,weight3],
        [weight4 ,weight3 , weight2 ,weight3 ,weight4]]

    convoluted_image = np.zeros(image.shape)

    window = [-2,-1,0,1,2]
    x_offset = 2
    y_offset = 2

    # Apply Convolution with Receptive Field Kernel
    for x_image_index in range(image_size_x):
        for y_image_index in range(image_size_y):
            summation = 0
            for x_kernel_index in window:
                for y_kernel_index in window:
                    if (x_image_index+x_kernel_index)>=0 and (x_image_index+x_kernel_index)<=image_size_x-1 and (y_image_index+y_kernel_index)>=0 and (y_image_index+y_kernel_index)<=image_size_y-1:
                        summation = summation + (receptive_field[x_offset+x_kernel_index][y_offset+y_kernel_index] * image[x_image_index+x_kernel_index][y_image_index+y_kernel_index]) / 255
            convoluted_image[x_image_index][y_image_index] = summation
    return convoluted_image


In [9]:
input = np.random.rand(255,255)
receptive_field(input)

array([[ 0.00266146,  0.00268965,  0.00094861, ...,  0.00525109,
         0.00685579,  0.00365209],
       [ 0.00576661,  0.00673441,  0.00488531, ...,  0.00563804,
         0.00761841,  0.00603703],
       [ 0.0059476 ,  0.00551406,  0.00396932, ...,  0.00244514,
         0.00512528,  0.00603047],
       ...,
       [ 0.00589134,  0.00912306,  0.0065426 , ..., -0.00061937,
         0.00399403,  0.00444065],
       [ 0.0034775 ,  0.00631703,  0.00562848, ...,  0.00118717,
         0.00463192,  0.00629945],
       [ 0.00011449,  0.00114841,  0.00202604, ...,  0.00144466,
         0.00423957,  0.00531666]])